In [1]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [2]:
URM_test=sps.load_npz("./dressipi_dataset/URM_test_leaderboard.npz")

In [3]:
URM_test.shape

(4439758, 23692)

In [4]:
np.ravel(URM_test.sum(axis=1)).shape

(4439758,)

In [5]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [6]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [7]:
is_sorted(Session_ids)

True

In [8]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"]
for file in ["UCF","ICBF","Graph","ICF", "TopPop15"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./leaderboard_ranker_dataset/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF
15952528
loaded UCF
0
loading ICBF
14870776
loaded ICBF
0
loading Graph
8315774
loaded Graph
0
loading ICF
17772188
loaded ICF
0
loading TopPop15
7500000
loaded TopPop15


In [9]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
map_ids["Real_Id"]=Session_ids

In [10]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [11]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [12]:
import ast
submission_df=pd.read_csv("./candidate-nn/leaderboard_candidates.csv")
submission_df["score"]=submission_df["score"].map(ast.literal_eval)
submission_df["code"]=submission_df["code"].map(ast.literal_eval)
submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
submission_df = submission_df.explode(['code','score', 'rank'])
submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [13]:
import ast
submission_df_tr=pd.read_csv("./candidate-transformer/leaderboard_candidates_transformer.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [14]:
import ast
submission_df_LSTM=pd.read_csv("./candidate-lstm/leaderboard_candidates_lstm_bidirectional.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [15]:
dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [16]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [17]:
dataset.fillna(0,inplace=True)

In [18]:
dataset_macro_features = pd.read_csv('./macro-feats/test_leaderboard_with_macro_feats.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [19]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [20]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Score_TopPop15,Session_Id,score_gru,rank_gru,score_transformer,...,Max_Score_TopPop15,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,21,0.100227,0.0,0.019740,0.000000,0.000000,26,0.000546,91.0,0.000000,...,5611.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,52,0.242711,0.0,0.087397,0.548803,0.000000,26,0.001073,40.0,0.003816,...,5611.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,80,0.090264,0.0,0.045723,0.545978,0.000000,26,0.000637,78.0,0.001016,...,5611.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,115,0.025982,0.0,0.000000,0.000000,0.142399,26,0.000000,0.0,0.000000,...,5611.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,159,0.058162,0.0,0.020845,0.000000,0.000000,26,0.000957,44.0,0.000000,...,5611.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [21]:
gc.collect()

1265

In [22]:
dataset=dataset.sort_values("Session_Id")

In [23]:
gc.collect()

0

In [24]:
import lightgbm

In [25]:
k=10

In [26]:
result = []

In [27]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'./{k}_fold_ranker_NN_transformer_LSTM_with_features_toppop_15/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))
    # dataset[f"score_{index}"]=model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1))

#combine scores
result = np.array(result)
result

array([[-1.51904362, -2.09138134, -2.23169096, ..., -3.26927944,
        -3.26927944, -2.9508289 ],
       [-1.23491813, -2.00225186, -1.89832085, ..., -2.81853125,
        -2.81853125, -2.48279298],
       [-1.5249909 , -2.12159753, -2.04335308, ..., -3.34668535,
        -3.34668535, -2.85213456],
       ...,
       [-1.51021759, -2.04985476, -1.99186448, ..., -3.53393884,
        -3.53393884, -3.0165997 ],
       [-1.61905155, -2.0509615 , -2.19745481, ..., -3.49132519,
        -3.49132519, -2.84485115],
       [-1.4515622 , -1.97450979, -1.96712669, ..., -3.53494641,
        -3.53494641, -3.02686599]])

In [28]:
result.shape

(10, 34953636)

In [29]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

#dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.median(x),axis=1)

In [30]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

0

In [31]:
dataset.head()

,Session_Id,Item_ID,score
0,26,21,-1.497498
28683093,26,1636,-2.056698
28683092,26,1609,-2.039639
28683091,26,1516,-1.992619
28683090,26,1497,-2.603780


In [32]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [33]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [34]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [35]:
dataset.iloc[0]

Session_Id                                                   26
Item_ID       [21, 1636, 1609, 1516, 1497, 1468, 1463, 1449,...
score         [-1.4974979953776768, -2.056697584733821, -2.0...
indices       [647, 700, 446, 433, 613, 708, 638, 453, 697, ...
Name: 0, dtype: object

In [36]:
dataset.iloc[0]["Item_ID"][536]

3406

In [37]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [38]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [39]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [40]:
dataset

,Session_Id,predictions,rank
0,26,"[606, 1008, 4952, 4685, 2905, 1205, 2296, 3894...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[772, 1827, 270, 4878, 3893, 3400, 3656, 3229,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[2289, 4986, 3398, 1181, 2075, 3842, 2235, 320...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[2378, 3937, 3337, 4262, 4900, 2369, 2022, 381...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[1004, 3172, 2175, 1768, 143, 25, 1165, 2485, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[2258, 353, 2605, 4810, 3526, 2800, 977, 2530,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[1875, 3880, 2877, 3026, 867, 3153, 1248, 3046...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[1901, 4779, 2638, 2431, 625, 4846, 2266, 1780...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[1015, 3531, 42, 2596, 989, 3665, 2266, 36, 36...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [41]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [42]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [43]:
dataset.columns=["session_id","code","rank"]

In [44]:
dataset.head()

,session_id,code,rank
0,26,"(606, 1008, 4952, 4685, 2905, 1205, 2296, 3894...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(772, 1827, 270, 4878, 3893, 3400, 3656, 3229,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(2289, 4986, 3398, 1181, 2075, 3842, 2235, 320...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(2378, 3937, 3337, 4262, 4900, 2369, 2022, 381...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(1004, 3172, 2175, 1768, 143, 25, 1165, 2485, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [45]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [46]:
mapping= pd.read_csv('./dressipi_dataset/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [47]:
print(len(submission_df))

5000000


In [48]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [49]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [50]:
submission_df.to_csv(
    f'predictions_leaderboard_{k}_fold_ranker_NN_transformer_LSTM_with_features_toppop_15_mean.csv',
    index=False
    )